## Ball Tracking with OpenCV


The goal here is fair self-explanatory:

    Step #1: Detect the presence of a colored ball using computer vision techniques.
    Step #2: Track the ball as it moves around in the video frames, drawing its previous positions as it moves.


In [1]:
from collections import deque
from imutils.video import VideoStream
import numpy as np
import argparse
import cv2
import imutils
import time

In [3]:
greenLower = (29, 86, 6)#range of green colour
greenUpper = (64, 255, 255)
pts = deque(maxlen=64) # list like data structure to store last location of ball in queue 

In [ ]:
vs = cv2.VideoCapture('ball_tracking_example.mp4')  #capturing video and reading frames from it 
while True:
    frame=vs.read()
    frame=frame[1]
    frame = imutils.resize(frame, width=600)
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)#reduce high frequency noise
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)#convert to hsv
    mask = cv2.inRange(hsv, greenLower, greenUpper) # a mask for green color
    mask = cv2.erode(mask, None, iterations=2)      #erosions and dilations to remove any small blobs left in mask
    mask = cv2.dilate(mask, None, iterations=2)
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)#finding contour in mask
    cnts = cnts[0] if imutils.is_cv2() else cnts[1] 
    center = None
    if(len(cnts)>0):
        # find the largest contour in the mask, then use it to compute the minimum enclosing circle and centroid
        c=max(cnts,key=cv2.contourArea)
        ((x,y),radius)=cv2.minEnclosingCircle(c)
        M = cv2.moments(c)#to find different features of contour like area,centroid,etc
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        
        if radius > 10:

            cv2.circle(frame, (int(x), int(y)), int(radius),
                (0, 255, 255), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)
 
    # update the points queue
    pts.appendleft(center)

    for i in range(1, len(pts)):# loop over the set of tracked points
        # if either of the tracked points are None, ignore them
        if pts[i - 1] is None or pts[i] is None:
            continue
 
        # otherwise, compute the thickness of the line and draw the connecting lines
        thickness = int(np.sqrt(64 / float(i + 1)) * 2.5)
        cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)
 
    # show the frame to our screen
    cv2.imshow('mask',mask)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
 
    # if the 'q' key is pressed, stop the loop
    if key == ord("q"):
        break
cv2.destroyAllWindows()        